In [ ]:
import tensorflow as tf
import json
# import keras
# from keras.layers import *
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import tensorflow as tf
from tensorflow.python.keras.layers import *
from tensorflow.python import keras

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model
# import tensorflow_text as text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
MAX_LEN = 100

In [ ]:
import json
blog_posts_data_dir = "/content/drive/MyDrive/datasets/BT-AP-19 Corpus/all/"
# blog_posts_data_dir = "/content/drive/MyDrive/datasets/"
train_file_name = "train.json"
test_file_name = "test.json"

# Load data
with open(blog_posts_data_dir + train_file_name) as r:
    training_set = json.load(r)
training_set = [i for i in training_set if len(i['post']) <= MAX_LEN]
len(training_set), training_set[0]

(25666,
 {'age': '50',
  'gender': 'female',
  'post': 'Umar Murtaza: Head of SM Operation'})

In [ ]:
def get_gender_as_num(gender):
    if gender == "male":
        return 0
    else:
        return 1

In [ ]:
def get_age_group(age): # HIGH NOTE: changing each of the scalars to a vector. This is probably not a good idea
    if age < 18:
        # 13 - 17
        return [1, 0, 0]
    elif age < 28:
        # 23 - 27
        return [0, 1, 0]
    elif age < 49:
        # 33 - 48
        return [0, 0, 1]
    else:
        return [0, 0, 0]

In [ ]:
def get_age_group(age): # HIGH NOTE: changing each of the scalars to a vector. This is probably not a good idea
    age = int(age)
    if age == 18:
        # 13 - 17
        return [1, 0, 0, 0]
    elif age == 25:
        # 23 - 27
        return [0, 1, 0, 0]
    elif age == 35:
        # 33 - 48
        return [0, 0, 1, 0]
    elif age == 50:
        # 33 - 48
        return [0, 0, 0, 1]
    else:
        return [0, 0, 0, 0]

In [ ]:
posts = [instance["post"] for instance in training_set]
len(posts)

25666

In [ ]:
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(posts)

In [ ]:
len(tk.word_index)
# alphabet = "abcdefghijklmnopqrstuvwxyz0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
# char_dict = {}
# for i, char in enumerate(alphabet):
#     char_dict[char] = i + 1

# Use char_dict to replace the tk.word_index
tk.word_index = char_dict.copy()
# Add 'UNK' to the vocabulary
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1
len(tk.word_index)

69

In [ ]:
len(tk.word_index)

440

In [ ]:
sequences = tk.texts_to_sequences(posts)

In [ ]:
data = keras.preprocessing.sequence.pad_sequences(sequences, maxlen = MAX_LEN,
                                                     padding = "post")
for i, instance in enumerate(training_set):
    instance["post"] = data[i]
    instance["gender"] = get_gender_as_num(instance["gender"])
    instance["age"] = get_age_group(int(instance["age"]))

In [ ]:
training_set[0]

{'age': [0, 0, 0, 1],
 'gender': 1,
 'post': array([13, 12,  4, 10,  2, 12, 13, 10,  5,  4, 30,  4, 26,  2, 11,  3,  4,
        18,  2,  6, 25,  2,  7, 12,  2,  6, 15,  3, 10,  4,  5,  9,  6,  8,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       dtype=int32)}

In [ ]:
EMBEDDING_DIM = 100
vocab_size = len(tk.word_index)

def create_model(type=True):
    model = keras.Sequential()
    model.add(keras.layers.Embedding(vocab_size + 1, EMBEDDING_DIM, input_length = MAX_LEN, trainable = True))


    # Add hidden layers 
    for i in range(0, 3):
        # Add a bidirectional lstm layer
        model.add(Bidirectional(LSTM(32, return_sequences=True, recurrent_dropout=0.2)))
        # Add a dropout layer after each lstm layer
        model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(32, recurrent_dropout=0.2)))
    model.add(Dropout(0.5))
    # Add the fully connected layer with 256 nurons and relu activation
    model.add(Dense(256, activation='relu'))
    # Add the output layer with softmax activation since we have 2 classes
    if type: 
        model.add(Dense(4, activation='softmax')) 
    else:
        model.add(Dense(1, activation='sigmoid')) 

    return model



In [ ]:
import os
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)
with strategy.scope():
    model = create_model()
    model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
    model.summary()

INFO:tensorflow:Initializing the TPU system: grpc://10.123.227.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.123.227.122:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 100)          44100     
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 100, 64)           34048     
_________________________________________________________________
dropout_12 (Dropout)         (None, 100, 64)           0         
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 100, 64)           24832     
_________________________________________________________________
dropout_13 (Dropout)         (None, 100, 64)           0         
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 100, 64)           24832     
_________________________________________________________________
dropout_14 (Dropout)         (None, 100, 64)          

In [ ]:
posts_train = np.array([instance["post"] for instance in training_set])
ages_train = np.array([instance["age"] for instance in training_set])

In [ ]:
history_1 = model.fit(posts_train, ages_train, epochs = 10, batch_size = 50, validation_split = 0.2)

Epoch 1/10
411/411 [==============================] - 24s 59ms/step - loss: 1.2547 - accuracy: 0.3650 - val_loss: 1.2474 - val_accuracy: 0.3935
Epoch 2/10
411/411 [==============================] - 24s 59ms/step - loss: 1.2419 - accuracy: 0.3890 - val_loss: 1.2127 - val_accuracy: 0.4104
Epoch 3/10
411/411 [==============================] - 24s 59ms/step - loss: 1.2391 - accuracy: 0.3867 - val_loss: 1.2346 - val_accuracy: 0.3878
Epoch 4/10
411/411 [==============================] - 24s 59ms/step - loss: 1.2231 - accuracy: 0.3967 - val_loss: 1.2106 - val_accuracy: 0.4147
Epoch 5/10
411/411 [==============================] - 24s 59ms/step - loss: 1.2123 - accuracy: 0.4113 - val_loss: 1.2016 - val_accuracy: 0.4254
Epoch 6/10
411/411 [==============================] - 24s 59ms/step - loss: 1.2032 - accuracy: 0.4228 - val_loss: 1.2051 - val_accuracy: 0.4172
Epoch 7/10
411/411 [==============================] - 24s 59ms/step - loss: 1.1983 - accuracy: 0.4177 - val_loss: 1.2007 - val_accuracy:

In [ ]:
with open(blog_posts_data_dir + test_file_name) as r:
    test_set = json.load(r)

In [ ]:
test_posts = [instance["post"] for instance in test_set]
test_sequences = tk.texts_to_sequences(test_posts)
test_post_data = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen = MAX_LEN,
                                                     padding = "post")
for i, instance in enumerate(test_set):
    instance["post"] = test_post_data[i]
    instance["gender"] = get_gender_as_num(instance["gender"])
    instance["age"] = get_age_group(int(instance["age"]))

In [ ]:
posts_test = np.array([instance["post"] for instance in test_set])
ages_test = np.array([instance["age"] for instance in test_set])

In [ ]:
model.evaluate(posts_test, ages_test)

263/263 [==============================] - 8s 24ms/step - loss: 1.2317 - accuracy: 0.4113


[1.231699824333191, 0.4112960696220398]

In [ ]:
from sklearn.metrics import classification_report
pred = model.predict(posts_test)
print(classification_report(np.argmax(ages_test, axis=1), np.argmax(pred, axis=1)))

              precision    recall  f1-score   support

           0       0.45      0.62      0.53      2698
           1       0.36      0.46      0.40      2647
           2       0.44      0.24      0.31      2309
           3       0.00      0.00      0.00       756

    accuracy                           0.41      8410
   macro avg       0.31      0.33      0.31      8410
weighted avg       0.38      0.41      0.38      8410



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Gender classification

In [ ]:
EMBEDDING_DIM = 100
vocab_size = len(tk.word_index)

def create_model(type=True):
    model = keras.Sequential()
    model.add(keras.layers.Embedding(vocab_size + 1, EMBEDDING_DIM, input_length = MAX_LEN, trainable = True))


    # Add hidden layers 
    for i in range(0, 3):
        # Add a bidirectional lstm layer
        model.add(Bidirectional(LSTM(32, return_sequences=True, recurrent_dropout=0.2)))
        # Add a dropout layer after each lstm layer
        model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(32, recurrent_dropout=0.2)))
    model.add(Dropout(0.5))
    # Add the fully connected layer with 256 nurons and relu activation
    model.add(Dense(256, activation='relu'))
    # Add the output layer with softmax activation since we have 2 classes
    
    model.add(Dense(1, activation='sigmoid')) 

    return model

In [ ]:
import os
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)
with strategy.scope():
    model2 = create_model()
    model2.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
    model2.summary()

INFO:tensorflow:Initializing the TPU system: grpc://10.123.227.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.123.227.122:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 100)          44100     
_________________________________________________________________
bidirectional_20 (Bidirectio (None, 100, 64)           34048     
_________________________________________________________________
dropout_20 (Dropout)         (None, 100, 64)           0         
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 100, 64)           24832     
_________________________________________________________________
dropout_21 (Dropout)         (None, 100, 64)           0         
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 100, 64)           24832     
_________________________________________________________________
dropout_22 (Dropout)         (None, 100, 64)          

In [ ]:
posts_train = np.array([instance["post"] for instance in training_set])
genders_train = np.array([instance["gender"] for instance in training_set])

In [ ]:
history_2 = model2.fit(posts_train, genders_train, epochs = 10, batch_size = 50, validation_split = 0.2)

Epoch 1/10
411/411 [==============================] - 63s 87ms/step - loss: 0.6435 - accuracy: 0.6624 - val_loss: 0.6243 - val_accuracy: 0.6677
Epoch 2/10
411/411 [==============================] - 24s 58ms/step - loss: 0.6217 - accuracy: 0.6677 - val_loss: 0.6399 - val_accuracy: 0.6675
Epoch 3/10
411/411 [==============================] - 24s 59ms/step - loss: 0.6369 - accuracy: 0.6694 - val_loss: 0.6352 - val_accuracy: 0.6685
Epoch 4/10
411/411 [==============================] - 24s 59ms/step - loss: 0.6330 - accuracy: 0.6734 - val_loss: 0.6343 - val_accuracy: 0.6687
Epoch 5/10
411/411 [==============================] - 24s 59ms/step - loss: 0.6300 - accuracy: 0.6719 - val_loss: 0.6171 - val_accuracy: 0.6677
Epoch 6/10
411/411 [==============================] - 24s 59ms/step - loss: 0.6135 - accuracy: 0.6698 - val_loss: 0.6104 - val_accuracy: 0.6663
Epoch 7/10
411/411 [==============================] - 24s 59ms/step - loss: 0.6103 - accuracy: 0.6714 - val_loss: 0.6136 - val_accuracy:

In [ ]:
posts_test = np.array([instance["post"] for instance in test_set])
genders_test = np.array([instance["gender"] for instance in test_set])

In [ ]:
model2.evaluate(posts_test, genders_test)

263/263 [==============================] - 8s 24ms/step - loss: 0.5841 - accuracy: 0.7002


[0.584140419960022, 0.7002378106117249]

In [ ]:
from sklearn.metrics import classification_report
pred = model2.predict(posts_test)
print(classification_report(genders_test, (pred > 0.5).astype(int)))

              precision    recall  f1-score   support

           0       0.67      0.03      0.06      2565
           1       0.70      0.99      0.82      5845

    accuracy                           0.70      8410
   macro avg       0.69      0.51      0.44      8410
weighted avg       0.69      0.70      0.59      8410

